In [44]:
!pip install tensorflow
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install matplotlib


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [20]:

covertype = fetch_ucirepo(id=31)

# Extract features (X) and targets (y)
X = covertype.data.features
y = covertype.data.targets
print("Dataset shape (features):", X.shape)
print("Dataset shape (targets):", y.shape)


print("\nFirst 5 rows of X:")
print(X.head())

print("\nFirst 5 rows of y:")
print(y.head())


Dataset shape (features): (581012, 54)
Dataset shape (targets): (581012, 1)

First 5 rows of X:
   Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  \
0       2596      51      3                               258   
1       2590      56      2                               212   
2       2804     139      9                               268   
3       2785     155     18                               242   
4       2595      45      2                               153   

   Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                               0                              510   
1                              -6                              390   
2                              65                             3180   
3                             118                             3090   
4                              -1                              391   

   Hillshade_9am  Hillshade_Noon  Hillshade_3pm  \
0            221             232         

In [21]:
# Convert y from a DataFrame/Series to a 1D array
y = y.values.ravel()

# Now shift labels so they become 0..6
y = y - 1

print("Unique y labels after shifting:", np.unique(y))


Unique y labels after shifting: [0 1 2 3 4 5 6]


In [43]:
print("Missing values per column in X:")
print(X.isnull().sum())


Missing values per column in X:
Elevation                             0
Aspect                                0
Slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Wilderness_Area1                      0
Soil_Type1                            0
Soil_Type2                            0
Soil_Type3                            0
Soil_Type4                            0
Soil_Type5                            0
Soil_Type6                            0
Soil_Type7                            0
Soil_Type8                            0
Soil_Type9                            0
Soil_Type10                           0
Soil_Type11                           0
Soil_Type12                           0
Soil_Type13                           0
Soil_Typ

In [23]:
from sklearn.preprocessing import StandardScaler

# List of numerical columns in Covertype
numerical_cols = [
    'Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
    'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points'
]

# Create a copy of X so we don't alter the original
X_scaled = X.copy()

scaler = StandardScaler()
X_scaled[numerical_cols] = scaler.fit_transform(X_scaled[numerical_cols])

# Inspect first 2 rows of scaled data to confirm
print("First 2 rows of scaled X:")
print(X_scaled.head(2))


First 2 rows of scaled X:
   Elevation    Aspect     Slope  Horizontal_Distance_To_Hydrology  \
0  -1.297805 -0.935157 -1.482820                         -0.053767   
1  -1.319235 -0.890480 -1.616363                         -0.270188   

   Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                       -0.796273                        -1.180146   
1                       -0.899197                        -1.257106   

   Hillshade_9am  Hillshade_Noon  Hillshade_3pm  \
0       0.330743        0.439143       0.142960   
1       0.293388        0.590899       0.221342   

   Horizontal_Distance_To_Fire_Points  ...  Soil_Type34  Soil_Type35  \
0                            3.246283  ...            0            0   
1                            3.205504  ...            0            0   

   Soil_Type36  Soil_Type37  Soil_Type38  Soil_Type39  Soil_Type40  \
0            0            0            0            0            0   
1            0            0            0 

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_scaled,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Training set:", X_train.shape, y_train.shape)
print("Validation set:", X_val.shape, y_val.shape)


Training set: (464809, 54) (464809,)
Validation set: (116203, 54) (116203,)


In [29]:
batch_size = 128

# If X_train is a DataFrame:
X_overfit = X_train.iloc[:batch_size].to_numpy()
# If y_train is already a NumPy array:
y_overfit = y_train[:batch_size]

# Convert validation sets to NumPy arrays if needed:
X_val_np = X_val.to_numpy()  # in case X_val is still a DataFrame
y_val_np = y_val  # if y_val is already a NumPy array

print("Shape of overfit subset:", X_overfit.shape, y_overfit.shape)


Shape of overfit subset: (128, 54) (128,)


In [30]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Number of features in our dataset:
input_dim = X_train.shape[1]

# 1. Input layer
inputs = Input(shape=(input_dim,))

# 2. First Dense layer
x = Dense(16, activation='relu')(inputs)

# 3. Second Dense layer
x = Dense(16, activation='relu')(x)


temp_model = Model(inputs=inputs, outputs=x)
temp_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 54)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 16)                  │             880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 16)                  │             272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,152 (4.50 KB)

 Trainable params: 1,152 (4.50 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
def residual_block(x, units=16):
    """
    A simple residual block with:
      -> Dense(units) + ReLU
      -> Dense(units) + ReLU
      -> plus skip connection
    """
    # 1) Check if dimension matches
    # If x.shape[-1] doesn't match 'units', we project x
    if x.shape[-1] != units:
        skip = Dense(units)(x)  # linear projection
    else:
        skip = x

    # 2) Main path: two Dense layers
    out = Dense(units, activation='relu')(x)
    out = Dense(units, activation='relu')(out)

    # 3) Add the skip
    out = out + skip
    return out


In [32]:
test_input = Input(shape=(16,))  # Suppose we test with dimension 16
test_output = residual_block(test_input, units=16)
test_model = Model(inputs=test_input, outputs=test_output)

test_model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 16)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 16)             │            272 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 16)             │            272 │ dense_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 16)             │              0 │ dense_10[0][0],        │
│                           │                        │                │ input_layer_2[0][0]    │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 544 (2.12 KB)

 Trainable params: 544 (2.12 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
# 1) Input layer
inputs = Input(shape=(input_dim,), name="my_input")

# 2) Two Dense layers
x = Dense(16, activation='relu', name="dense_initial_1")(inputs)
x = Dense(16, activation='relu', name="dense_initial_2")(x)


skip_connection = Dense(16, name="skip_connection_dense")(inputs)

# 3) Apply the residual block
x = residual_block(x, units=16)

# 4) skip
x = x + skip_connection


temp_model_2 = Model(inputs=inputs, outputs=x)
temp_model_2.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ my_input (InputLayer)     │ (None, 54)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_initial_1 (Dense)   │ (None, 16)             │            880 │ my_input[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_initial_2 (Dense)   │ (None, 16)             │            272 │ dense_initial_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 16)             │            272 │ dense_initial_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 16)             │            272 │ dense_11[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_3 (Add)               │ (None, 16)             │              0 │ dense_12[0][0],        │
│                           │                        │                │ dense_initial_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ skip_connection_dense     │ (None, 16)             │            880 │ my_input[0][0]         │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_4 (Add)               │ (None, 16)             │              0 │ add_3[0][0],           │
│                           │                        │                │ skip_connection_dense… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,576 (10.06 KB)

 Trainable params: 2,576 (10.06 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
from tensorflow.keras.optimizers import Adam



final_x = Dense(16, activation='relu', name="dense_post_skip")(temp_model_2.output)


outputs = Dense(7, activation='softmax', name="output_layer")(final_x)


model = Model(inputs=temp_model_2.input, outputs=outputs)

#  Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ my_input (InputLayer)     │ (None, 54)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_initial_1 (Dense)   │ (None, 16)             │            880 │ my_input[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_initial_2 (Dense)   │ (None, 16)             │            272 │ dense_initial_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 16)             │            272 │ dense_initial_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 16)             │            272 │ dense_11[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_3 (Add)               │ (None, 16)             │              0 │ dense_12[0][0],        │
│                           │                        │                │ dense_initial_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ skip_connection_dense     │ (None, 16)             │            880 │ my_input[0][0]         │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_4 (Add)               │ (None, 16)             │              0 │ add_3[0][0],           │
│                           │                        │                │ skip_connection_dense… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_post_skip (Dense)   │ (None, 16)             │            272 │ add_4[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ output_layer (Dense)      │ (None, 7)              │            119 │ dense_post_skip[0][0]  │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,967 (11.59 KB)

 Trainable params: 2,967 (11.59 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
model.save("my_model.keras")


In [37]:
epochs = 200  # or more if needed
history = model.fit(
    X_overfit,
    y_overfit,
    epochs=epochs,
    batch_size=128,
    validation_data=(X_val_np, y_val_np),
    verbose=1
)


Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.9922 - loss: 0.1088 - val_accuracy: 0.6161 - val_loss: 1.7228
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 1.0000 - loss: 0.1068 - val_accuracy: 0.6161 - val_loss: 1.7337
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 1.0000 - loss: 0.1049 - val_accuracy: 0.6159 - val_loss: 1.7448
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 1.0000 - loss: 0.1029 - val_accuracy: 0.6159 - val_loss: 1.7558
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 1.0000 - loss: 0.1010 - val_accuracy: 0.6158 - val_loss: 1.7667
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 1.0000 - loss: 0.0991 - val_accuracy: 0.6157 - val_loss: 1.7776
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 1.0000 - loss: 0.0973 - val_accuracy: 0.6155 - val_loss: 1.7891
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 1.0000 - loss: 0.0955 - val_accuracy: 0.6152 - val_loss: 1.8007
Epoch 9/

In [42]:
train_loss, train_acc = model.evaluate(X_overfit, y_overfit, verbose=0)
val_loss, val_acc = model.evaluate(X_val_np, y_val_np, verbose=0)

print("\n========== Conclusions ==========")
print("Number of parameters:", model.count_params())
print("Final training loss:", train_loss)
print("Final validation loss:", val_loss)
print("=================================")



========== Conclusions ==========
Number of parameters: 2967
Final training loss: 0.006128049921244383
Final validation loss: 3.296598196029663


In [46]:
from google.colab import files
files.download('my_model.keras')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>